In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn import preprocessing as pre
import pandas as pd
import matplotlib.pyplot as plt

# DATA PREPARATION

In [4]:
from api_helpers.team_stats_helpers import load_dataframe
import pandas as pd

nba_dataframe = load_dataframe(['FGM', 'FGA', 'FG_PCT', 'FG3A', 'FTM', 'OREB', 'DREB', 'REB', 'AST','PTS'])
nba_dataframe = nba_dataframe.drop(nba_dataframe[nba_dataframe["FGA"] == 0].index)

nba_dataframe["YEAR"] = "2" + nba_dataframe["YEAR"].str.slice(0,4)
pd.DataFrame.rename(nba_dataframe, columns={"YEAR": "SEASON_ID"}, inplace=True)

nba_dataframe["NBA_FINALS_APPEARANCE"].fillna(0.0, inplace=True)
nba_dataframe["NBA_FINALS_APPEARANCE"].replace("FINALS APPEARANCE", 0, inplace=True)
nba_dataframe["NBA_FINALS_APPEARANCE"].replace("LEAGUE CHAMPION", 1, inplace=True)

dataframe_2023 = nba_dataframe[nba_dataframe['SEASON_ID'] == "22023"]
nba_dataframe = nba_dataframe[nba_dataframe['SEASON_ID'] != "22023"]
nba_dataframe = nba_dataframe.reset_index(drop=True)

/tmp/ipykernel_11889/3578733798.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nba_dataframe["NBA_FINALS_APPEARANCE"].fillna(0.0, inplace=True)
/tmp/ipykernel_11889/3578733798.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nba_dataframe["NBA_FINALS_APPEARANCE"].replace("LEAGUE CHAMPION",

In [5]:
# # best features: ['FGA', 'FG_PCT', 'FG3M', 'FTM', 'FT_PCT', 'DREB', 'REB', 'PTS']
non_int_columns = ["TEAM_ID","SEASON_ID","NBA_FINALS_APPEARANCE"]
stats_frame = nba_dataframe.drop(non_int_columns, axis=1)
nameless_dataframe_2023 = dataframe_2023.drop(non_int_columns,axis=1)

In [6]:
from scipy.spatial import distance

def closest_teams(vectors_frame, k = 1, target = dataframe_2023.sample(1)):

    vectors = np.array(vectors_frame.drop(non_int_columns,axis=1))

    distances = distance.cdist(target, vectors, "cosine")[0]
    # Sort distances (indices of closest points at the beginning)
    closest_indices = np.argsort(distances)

    # take top k closest vectors
    return vectors_frame.iloc[list(closest_indices[:k])]

In [7]:
team_1_id = 1610612740
team_2_id = 1610612763

team_1_row = dataframe_2023[dataframe_2023["TEAM_ID"] == team_1_id]
team_2_row = dataframe_2023[dataframe_2023["TEAM_ID"] == team_2_id]

similar_rows_1 = closest_teams(nba_dataframe, k=10, target=team_1_row.drop(non_int_columns,axis=1))
similar_rows_1

,TEAM_ID,SEASON_ID,NBA_FINALS_APPEARANCE,FGM,FGA,FG_PCT,FG3A,FTM,OREB,DREB,REB,AST,PTS
1081,1610612764,22022,0.0,3456,7127,0.485,2601,1442,774,2804,3578,2083,9279
122,1610612739,22022,0.0,3408,6984,0.488,2589,1439,795,2574,3369,2045,9205
540,1610612750,22022,0.0,3515,7167,0.490,2731,1467,749,2686,3435,2145,9494
264,1610612743,22020,0.0,3114,6422,0.485,2462,1129,758,2442,3200,1933,8284
580,1610612751,22021,0.0,3442,7251,0.475,2602,1434,844,2796,3640,2071,9258
776,1610612756,22019,0.0,3006,6429,0.468,2320,1451,712,2465,3177,1987,8294
143,1610612740,22022,0.0,3447,7180,0.480,2468,1585,866,2717,3583,2122,9378
266,1610612743,22022,1.0,3574,7088,0.504,2559,1378,828,2699,3527,2368,9495
389,1610612746,22022,0.0,3370,7059,0.477,2735,1533,803,2742,3545,1959,9314
967,1610612761,22018,1.0,3460,7305,0.474,2771,1449,786,2920,3706,2085,9384


In [8]:
def join_teams():
    joined_list = []
    for _, row in similar_rows_1.iterrows():
        k=5
        year = row['SEASON_ID']

        # get 
        year_frame = nba_dataframe[nba_dataframe['SEASON_ID'] == year]

        similar_rows_2 = closest_teams(year_frame, k=k, target=team_2_row.drop(non_int_columns,axis=1))
        similar_rows_2 = similar_rows_2.add_suffix("_B")

        combined_row = pd.concat([row.to_frame().T] * len(similar_rows_2),axis=0)

        # Now merge the repeated_df1 with df2
        result = pd.concat([combined_row.reset_index(drop=True), similar_rows_2.reset_index(drop=True)], axis=1)

        joined_list.append(result)


    final_joined = pd.concat(joined_list, ignore_index=True)

    return final_joined

joined = join_teams()
# joined.drop(["SEASON_ID_B"],inplace=True,axis=1)
# pd.DataFrame.rename(joined, columns={"SEASON_ID_A":"SEASON_ID"},inplace=True)
joined

,TEAM_ID,SEASON_ID,NBA_FINALS_APPEARANCE,FGM,FGA,FG_PCT,FG3A,FTM,OREB,DREB,...,FGM_B,FGA_B,FG_PCT_B,FG3A_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B,PTS_B
0,1610612764,22022,0.0,3456,7127,0.485,2601,1442,774,2804,...,3485,7365,0.473,3099,1536,968,2794,3762,2129,9600
1,1610612764,22022,0.0,3456,7127,0.485,2601,1442,774,2804,...,3504,7411,0.473,3306,1364,914,3074,3988,2115,9589
2,1610612764,22022,0.0,3456,7127,0.485,2601,1442,774,2804,...,3385,7413,0.457,2669,1447,901,2751,3652,2062,9098
3,1610612764,22022,0.0,3456,7127,0.485,2601,1442,774,2804,...,3215,6991,0.460,2852,1567,796,2533,3329,1955,8977
4,1610612764,22022,0.0,3456,7127,0.485,2601,1442,774,2804,...,3585,7551,0.475,2807,1432,988,2835,3823,2135,9587
5,1610612739,22022,0.0,3408,6984,0.488,2589,1439,795,2574,...,3485,7365,0.473,3099,1536,968,2794,3762,2129,9600
6,1610612739,22022,0.0,3408,6984,0.488,2589,1439,795,2574,...,3504,7411,0.473,3306,1364,914,3074,3988,2115,9589
7,1610612739,22022,0.0,3408,6984,0.488,2589,1439,795,2574,...,3385,7413,0.457,2669,1447,901,2751,3652,2062,9098
8,1610612739,22022,0.0,3408,6984,0.488,2589,1439,795,2574,...,3215,6991,0.460,2852,1567,796,2533,3329,1955,8977
9,1610612739,22022,0.0,3408,6984,0.488,2589,1439,795,2574,...,3585,7551,0.475,2807,1432,988,2835,3823,2135,9587


In [10]:
joined.to_csv("new_data.csv")

In [10]:
all_games_df = pd.read_csv("data/all_games.csv")

In [11]:
from api_helpers.game_stats_helpers import home_matchups
final = pd.DataFrame()
final_stats_list = []

for i in range(len(joined)):
    row = joined.iloc[i]
    
    matchups = home_matchups(all_games_df=all_games_df,team_a=int(row["TEAM_ID"]), team_b=int(row["TEAM_ID_B"]), year=int(row["SEASON_ID"]))
    
    # output data
    final = pd.concat([final, matchups],axis=0)

    if len(matchups) == 0:
        continue
    else:
        repeated_row = pd.concat([row.to_frame().T] * len(matchups),axis=0)
        final_stats_list.append(repeated_row)

# dataframe of team stats (input data)
final_stats_df = pd.concat(final_stats_list,axis=0)

In [12]:
final.to_csv("test_data.csv")
type(all_games_df["TEAM_ID_B"].iloc[0])

numpy.int64

In [14]:
final.columns

Index(['Unnamed: 0', 'SEASON_ID', 'TEAM_ID_A', 'TEAM_ABBREVIATION_A',
       'TEAM_NAME_A', 'GAME_ID', 'GAME_DATE', 'MATCHUP_A', 'WL_A', 'MIN_A',
       'PTS_A', 'FGM_A', 'FGA_A', 'FG_PCT_A', 'FG3M_A', 'FG3A_A', 'FG3_PCT_A',
       'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A', 'DREB_A', 'REB_A', 'AST_A',
       'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PLUS_MINUS_A', 'TEAM_ID_B',
       'TEAM_ABBREVIATION_B', 'TEAM_NAME_B', 'MATCHUP_B', 'WL_B', 'MIN_B',
       'PTS_B', 'FGM_B', 'FGA_B', 'FG_PCT_B', 'FG3M_B', 'FG3A_B', 'FG3_PCT_B',
       'FTM_B', 'FTA_B', 'FT_PCT_B', 'OREB_B', 'DREB_B', 'REB_B', 'AST_B',
       'STL_B', 'BLK_B', 'TOV_B', 'PF_B', 'PLUS_MINUS_B'],
      dtype='object')

# Random Forest Modelling

In [13]:
# Getting input features
input_a_features = ['FG_PCT','FG3A', 'FTM', 'OREB', 'DREB', 'REB', 'AST']
input_b_features = [str(word) + "_B" for word in input_a_features]

input_features = input_a_features + input_b_features
X = final_stats_df[input_features]
X

,FG_PCT,FG3A,FTM,OREB,DREB,REB,AST,FG_PCT_B,FG3A_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B
0,0.485,2601,1442,774,2804,3578,2083,0.473,3099,1536,968,2794,3762,2129
1,0.485,2601,1442,774,2804,3578,2083,0.473,3306,1364,914,3074,3988,2115
1,0.485,2601,1442,774,2804,3578,2083,0.473,3306,1364,914,3074,3988,2115
2,0.485,2601,1442,774,2804,3578,2083,0.457,2669,1447,901,2751,3652,2062
2,0.485,2601,1442,774,2804,3578,2083,0.457,2669,1447,901,2751,3652,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,0.474,2771,1449,786,2920,3706,2085,0.447,3002,1541,832,2884,3716,1918
48,0.474,2771,1449,786,2920,3706,2085,0.449,2965,1555,900,2919,3819,1954
48,0.474,2771,1449,786,2920,3706,2085,0.449,2965,1555,900,2919,3819,1954
49,0.474,2771,1449,786,2920,3706,2085,0.465,2829,1282,804,2849,3653,2155


In [14]:
output_a_features = ["FG_PCT_A","FG3_PCT_A","FTM_A","OREB_A","DREB_A","REB_A","AST_A"]
output_b_features = [word.replace("_A","_B") for word in output_a_features]
output_features = output_a_features + output_b_features
y = final[output_features]
y

,FG_PCT_A,FG3_PCT_A,FTM_A,OREB_A,DREB_A,REB_A,AST_A,FG_PCT_B,FG3_PCT_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B
47753,0.500,0.444,17,6.0,40.0,46.0,27,0.468,0.350,10,7.0,35.0,42.0,33
47721,0.505,0.481,15,9.0,33.0,42.0,30,0.543,0.548,17,12.0,44.0,56.0,33
47730,0.426,0.258,11,20.0,36.0,56.0,21,0.470,0.449,17,7.0,38.0,45.0,30
47735,0.506,0.405,15,11.0,41.0,52.0,29,0.419,0.290,17,12.0,28.0,40.0,23
47749,0.430,0.382,13,14.0,35.0,49.0,19,0.390,0.250,16,15.0,36.0,51.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43442,0.484,0.407,17,12.0,38.0,50.0,23,0.413,0.400,19,10.0,34.0,44.0,22
43419,0.543,0.290,18,5.0,35.0,40.0,32,0.523,0.488,13,5.0,26.0,31.0,33
43425,0.457,0.314,15,13.0,42.0,55.0,32,0.420,0.367,20,9.0,45.0,54.0,20
43415,0.489,0.472,9,8.0,37.0,45.0,33,0.384,0.200,23,10.0,37.0,47.0,24


### Creating a Test Train Split

In [15]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

### Implementing Random Forest Regressor

In [16]:
from sklearn.ensemble import RandomForestRegressor

rfe = RandomForestRegressor(random_state=10, n_estimators=1000)

rfe.fit(X_train,y_train)
y_pred = rfe.predict(X_test)
y_pred

array([[ 0.51814988,  0.36006645, 13.794725  , 10.945075  , 34.74644167,
        45.69151667, 29.26054167,  0.40575749,  0.24741276, 19.02954167,
        10.84059167, 23.17818333, 34.018775  , 18.79306667],
       [ 0.4572126 ,  0.31802557, 13.99808333,  6.26956667, 32.5173    ,
        38.78686667, 19.64061667,  0.40943085,  0.27333438, 21.0656    ,
         9.56595   , 29.4452    , 39.01115   , 17.24736667],
       [ 0.47203366,  0.39636956, 16.33096667, 11.85297778, 33.97944444,
        45.83242222, 26.82951111,  0.47887244,  0.33531214, 16.97690556,
        11.08633889, 34.18017222, 45.26651111, 28.19560556],
       [ 0.48649719,  0.34710795, 20.94682143, 10.13680952, 34.63811905,
        44.77492857, 26.53684524,  0.45533885,  0.32914892, 19.24490476,
         9.15536905, 33.58919048, 42.74455952, 23.31607143],
       [ 0.50813097,  0.38186037, 17.75448333,  9.64289167, 35.16599167,
        44.80888333, 28.69651667,  0.44059188,  0.33199813, 18.61043333,
        10.49784167, 29.87

In [25]:

y_pred = rfe.predict(X_test.loc[8].to_frame().T)
y_pred

array([[ 0.51814988,  0.36006645, 13.794725  , 10.945075  , 34.74644167,
        45.69151667, 29.26054167,  0.40575749,  0.24741276, 19.02954167,
        10.84059167, 23.17818333, 34.018775  , 18.79306667]])

## Evaluate Multioutput Regression

In [20]:
# evaluate multioutput regression model with k-fold cross-validation
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

# define model
model = RandomForestRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 3.589 (0.413)


## Creating Neural Network

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

recorded_games = all_games_df[all_games_df["WL_A"] != " "]

X_input = np.asarray(recorded_games[output_features])
y_output = recorded_games["WL_A"]

y_output.replace("L", 0, inplace=True)
y_output.replace("W", 1, inplace=True)

X_input = np.asarray(X_input).astype(np.float32)
y_output = np.asarray(y_output).astype(np.float32)

recorded_games[output_features + ["WL_A"]]

/tmp/ipykernel_305486/1318040735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_output.replace("L", 0, inplace=True)
/tmp/ipykernel_305486/1318040735.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_output.replace("W", 1, inplace=True)
/tmp/ipykernel_305486/1318040735.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_output.replace("W", 1, inplace=True)


,FG_PCT_A,FG3_PCT_A,FTM_A,OREB_A,DREB_A,REB_A,AST_A,FG_PCT_B,FG3_PCT_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B,WL_A
0,0.524,0.450,10,7.0,31.0,38.0,35,0.506,0.389,21,5.0,28.0,33.0,25,0.0
1,0.398,0.217,11,17.0,42.0,59.0,28,0.477,0.400,15,5.0,41.0,46.0,29,0.0
2,0.544,0.342,10,6.0,42.0,48.0,36,0.457,0.333,17,8.0,29.0,37.0,24,1.0
3,0.457,0.400,15,9.0,30.0,39.0,24,0.551,0.286,28,6.0,34.0,40.0,27,0.0
4,0.490,0.341,6,17.0,36.0,53.0,24,0.479,0.389,18,9.0,34.0,43.0,28,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52802,0.526,0.500,21,9.0,21.0,30.0,23,0.540,NaN,26,14.0,28.0,42.0,25,0.0
52803,0.436,0.000,11,16.0,28.0,44.0,20,0.442,NaN,23,15.0,34.0,49.0,18,0.0
52804,0.413,0.500,21,21.0,24.0,45.0,25,0.517,0.333,14,16.0,30.0,46.0,22,0.0
52805,0.478,0.000,31,17.0,38.0,55.0,28,0.482,0.111,22,13.0,35.0,48.0,21,1.0


## Creating Test Train Split

In [64]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_input,y_output,test_size=0.2,random_state=3)

## Building Model

In [1]:
from tensorflow.keras.models import model_from_json

# load json and create model
json_file = open("tuned_nn.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("tuned.weights.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

2024-07-12 16:08:55.190842: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 16:08:55.191308: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 16:08:55.192921: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 16:08:55.198658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 16:08:55.209895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

Loaded model from disk


2024-07-12 16:08:56.234905: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-12 16:08:56.234917: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: rohanbendapudi-Precision-3581
2024-07-12 16:08:56.234920: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: rohanbendapudi-Precision-3581
2024-07-12 16:08:56.234972: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-07-12 16:08:56.234979: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.183.1
2024-07-12 16:08:56.234980: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 535.183.1


In [3]:
predictions = loaded_model.predict()

print("Test loss:", predictions[0])
print("Test accuracy:", predictions[1])

NameError: name 'X_test' is not defined

In [104]:
_, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(accuracy)

ValueError: You must call `compile()` before using the model.

In [13]:
from api_helpers.three_step_pipeline import Matchup_Regressor

team_1_id = 1610612740
team_2_id = 1610612763

Matchup_Regressor(team_1_id, team_2_id)

/home/rohanbendapudi/github_repos/nba_predictor_development/models/api_helpers/three_step_pipeline.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nba_dataframe["NBA_FINALS_APPEARANCE"].fillna(0.0, inplace=True)
/home/rohanbendapudi/github_repos/nba_predictor_development/models/api_helpers/three_step_pipeline.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future b

<class 'pandas.core.frame.DataFrame'>
       TEAM_ID SEASON_ID NBA_FINALS_APPEARANCE   FGM   FGA FG_PCT  FG3A   FTM  \
0   1610612764     22022                   0.0  3456  7127  0.485  2601  1442   
1   1610612764     22022                   0.0  3456  7127  0.485  2601  1442   
1   1610612764     22022                   0.0  3456  7127  0.485  2601  1442   
2   1610612764     22022                   0.0  3456  7127  0.485  2601  1442   
2   1610612764     22022                   0.0  3456  7127  0.485  2601  1442   
..         ...       ...                   ...   ...   ...    ...   ...   ...   
47  1610612761     22018                   1.0  3460  7305  0.474  2771  1449   
48  1610612761     22018                   1.0  3460  7305  0.474  2771  1449   
48  1610612761     22018                   1.0  3460  7305  0.474  2771  1449   
49  1610612761     22018                   1.0  3460  7305  0.474  2771  1449   
49  1610612761     22018                   1.0  3460  7305  0.474  2771